In [ ]:
import os
os.chdir("C:/Users/Vinodh Chennu/tripadvisor")
from bs4 import BeautifulSoup
# for automation of clicks
from selenium import webdriver
import pandas as pd
import requests

In [ ]:
def get_code_and_url():
    hot_code = input("enter hotel code :")
    url = input("enter url :")
    return hot_code, url

def get_data_offset(url):
    # fetching html
    data = requests.get(url)
    # converting the html code into soup
    soup = BeautifulSoup(data.content,'html.parser')
    # extracting only reviews part
    a = soup.findAll("a", class_ = "pageNum")
    return int(a[-1].get_text())

def links_generator(url):
    url_=[]
    url_.append([1,1,url])
    data_offset = get_data_offset(url)*5
    for i in range(5,data_offset,5):
        url_.append([int(i/5)+1,int(i+1),url.replace("Reviews","Reviews-or"+str(i))])
        df = pd.DataFrame(url_, columns=['index','start_offset','url'])
        df.set_index('index', inplace=True)
    return df

def get_df(review_soup):
    id1=[]
    loc=[]
    date=[]
    slug=[]
    rating=[]
    review=[]
    mobile=[]
    hotel_reply=[]
    reply_content=[]
    # iterating over each review block
    for i in review_soup:
        id1.append(i.get("data-reviewid"))
    
        # finding the location
        loc_=i.find_all('span', class_='default social-member-common-MemberHometown__hometown--3kM9S small')
        if loc_ !=[]:
            loc.append(loc_[0].get_text())
        else:
            loc.append(None)
    
        # finding review date
        date_=i.find_all('div', class_='social-member-event-MemberEventOnObjectBlock__event_type--3njyv')
        a,b=date_[0].get_text().split(" ")[-2:]
        date.append(str(a+' '+b))
    
        # finding slug lines
        slug_ = i.find_all('div', class_='hotels-review-list-parts-ReviewTitle__reviewTitle--2Fauz')
        slug.append(slug_[0].get_text())
    
        rating_= i.find_all('div', class_='hotels-review-list-parts-RatingLine__bubbles--1oCI4')
        rating.append(rating_[0].find('span').get('class')[1][-2:-1])
    
        review.append(i.find('q', class_='hotels-review-list-parts-ExpandableReview__reviewText--3oMkH').find('span').get_text())
    
        reply_=i.find_all('div', class_='common-text-ReadMore__content--2X4LR')
        if len(reply_)>1:
            hotel_reply.append("yes")
            reply_content.append(reply_[0].find('span').get_text())
        else:
            hotel_reply.append("no")
            reply_content.append(None)
    
    frame=pd.DataFrame()
    frame['user_id'] = id1
    frame['review_date'] = date
    frame['user_location']=loc
    frame['review_slug']=slug
    frame['rating']=rating
    frame['review']=review
    frame['hotel_reply']=hotel_reply
    frame['reply_content']=reply_content
    return frame

def get_html(url):
    try:
        # needs a driver file, placed in envitoment variables PATH
        # starts a browser session
        driver = webdriver.Edge()
        # gets the html code and displays in the browser
        driver.get(url)
        # internal, searches for the more tag word
        elem = driver.find_element_by_class_name("hotels-review-list-parts-ExpandableReview__cta--3U9OU")
        # clicks on it, one click is enough to reveal all reviews
        elem.click()
        return driver.page_source
    except:
        data = requests.get(url)
        return data.content

def get_review_df(url, start_offset):
    html = get_html(url)
    # converting the html code into soup
    soup = BeautifulSoup(html,'html.parser')
    # extracting only reviews part
    review_soup = soup.find_all('div',class_='hotels-review-list-parts-SingleReview__reviewContainer--d54T4')
    df = get_df(review_soup)
    df.index = range(start_offset, start_offset+df.shape[0])
    return df

hot_code, url = get_code_and_url()
links = links_generator(url)
pd.DataFrame(columns=['user_id','review_date','user_location','review_slug','rating','review','hotel_reply',"reply_content"]).to_csv(hot_code+".csv")
for i in links.index:
    url = links['url'][i]
    start_offset = links['start_offset'][i]
    df = get_review_df(url, start_offset)
    df.to_csv(hot_code+".csv", mode='a', header=False)